### Quest
- BreastCancerWisconsinDataSet.csv 연속형 데이터 사용
- 목표변수(diagnosis), 설명변수(*_mean이 아닌 것 선택 3개)
- diagnosis 전처리 : apply() 이용<M-0, B-1>
    * 컴퓨터가 이해할 수 있는 이진으로 넣어주기 위해서 전처리 해줌
- Logistic regression 학습, 평가, 서비스(pickle)

### Quest(8/29일)
- train_test_split() 적용
- train과 test set 대한 classification_report() 적용

#### 변수 설정
1. 목표변수 : dianosis(진단결과) - 양성(B) 또는 악성(M)
2. 설명변수
    * radius_worst(최대 반경 크기)
    * perimeter_worst(최대 둘레 길이)
    * area_worst(최대 면적)

In [33]:
import pandas as pd

In [34]:
df_BCW = pd.read_csv('../../../datasets/BreastCancerWisconsinDataSet.csv')
df_BCW[:2]

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.8,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.6,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.9,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.8,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN


In [35]:
df_BCW.columns

Index(['id', 'diagnosis', 'radius_mean', 'texture_mean', 'perimeter_mean',
       'area_mean', 'smoothness_mean', 'compactness_mean', 'concavity_mean',
       'concave points_mean', 'symmetry_mean', 'fractal_dimension_mean',
       'radius_se', 'texture_se', 'perimeter_se', 'area_se', 'smoothness_se',
       'compactness_se', 'concavity_se', 'concave points_se', 'symmetry_se',
       'fractal_dimension_se', 'radius_worst', 'texture_worst',
       'perimeter_worst', 'area_worst', 'smoothness_worst',
       'compactness_worst', 'concavity_worst', 'concave points_worst',
       'symmetry_worst', 'fractal_dimension_worst', 'Unnamed: 32'],
      dtype='object')

In [36]:
df_BCW.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 33 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       569 non-null    int64  
 1   diagnosis                569 non-null    object 
 2   radius_mean              569 non-null    float64
 3   texture_mean             569 non-null    float64
 4   perimeter_mean           569 non-null    float64
 5   area_mean                569 non-null    float64
 6   smoothness_mean          569 non-null    float64
 7   compactness_mean         569 non-null    float64
 8   concavity_mean           569 non-null    float64
 9   concave points_mean      569 non-null    float64
 10  symmetry_mean            569 non-null    float64
 11  fractal_dimension_mean   569 non-null    float64
 12  radius_se                569 non-null    float64
 13  texture_se               569 non-null    float64
 14  perimeter_se             5

#### 전처리

In [37]:
# 목표/설명변수 추출, null값 확인
df_BCW_extract = df_BCW[['diagnosis','radius_worst','perimeter_worst','area_worst']]
df_BCW_extract.isnull().sum()

diagnosis          0
radius_worst       0
perimeter_worst    0
area_worst         0
dtype: int64

In [38]:
# diagnosis의 결과를 apply()를 이용하여 숫자로 변경 (M-0, B-1)
def diagnosis_result (cell) :
    if (cell == 'M') :
        result = 0
    elif (cell == 'B') :
        result = 1
    return result

In [39]:
df_BCW_extract['diagnosis_result'] = df_BCW_extract['diagnosis'].apply(diagnosis_result)
df_BCW_extract['diagnosis_result'][:2]

C:\Users\koreavc\AppData\Local\Temp\ipykernel_18044\196040931.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_BCW_extract['diagnosis_result'] = df_BCW_extract['diagnosis'].apply(diagnosis_result)


0    0
1    0
Name: diagnosis_result, dtype: int64

In [40]:
df_BCW_extract[:2]

,diagnosis,radius_worst,perimeter_worst,area_worst,diagnosis_result
0,M,25.38,184.6,2019.0,0
1,M,24.99,158.8,1956.0,0


#### 정형화 단계

In [41]:
from sklearn.model_selection import train_test_split
target = df_BCW_extract['diagnosis_result']
features = df_BCW_extract[['radius_worst','perimeter_worst','area_worst']]

In [42]:
features_train, features_test, target_train, target_test = train_test_split(features, target)
features_train.shape, target_train.shape, features_test.shape, target_test.shape

((426, 3), (426,), (143, 3), (143,))

In [43]:
target_train = df_BCW_extract['diagnosis_result']
features_train = df_BCW_extract[['radius_worst','perimeter_worst','area_worst']]
target_train.shape, features_train.shape

((569,), (569, 3))

#### 모델학습

In [44]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression()
model.fit(features, target)

LogisticRegression()

In [45]:
# 모델 일반화 확인
model.coef_, model.intercept_

(array([[ 0.41798351, -0.21153931, -0.00159987]]), array([17.67925737]))

#### 예측

In [46]:
df_BCW_extract[10:15]

,diagnosis,radius_worst,perimeter_worst,area_worst,diagnosis_result
10,M,19.19,123.8,1150.0,0
11,M,20.42,136.5,1299.0,0
12,M,20.96,151.7,1332.0,0
13,M,16.84,112.0,876.5,0
14,M,15.03,108.8,697.7,0


In [47]:
model.predict(features[10:15])
# 실제 데이터 값 : [0, 0, 0, 0, 0]
# 결과 : array([0, 0, 0, 0, 0]

array([0, 0, 0, 0, 0], dtype=int64)

In [48]:
# 확률
model.predict_proba(features[10:15])

# [0일 확률, 1일 확률]
# [9.11173751e-01, 8.88262494e-02]

array([[9.11173751e-01, 8.88262494e-02],
       [9.91327269e-01, 8.67273149e-03],
       [9.99582723e-01, 4.17277004e-04],
       [5.93049297e-01, 4.06950703e-01],
       [5.42436117e-01, 4.57563883e-01]])

#### 평가

In [49]:
target_train_predict = model.predict(features_train)
target_train_predict.shape

(569,)

In [50]:
from sklearn.metrics import accuracy_score # 정확도

In [51]:
accuracy_score(target_train, target_train_predict)  #교내시험

0.9191564147627417

In [52]:
target_test_predict = model.predict(features_test)
target_test_predict.shape

(143,)

In [53]:
accuracy_score(target_test, target_test_predict)   #교외시험

0.9020979020979021

In [54]:
from sklearn.metrics import classification_report

In [55]:
print(classification_report(target_train, target_train_predict))

              precision    recall  f1-score   support

           0       0.91      0.87      0.89       212
           1       0.92      0.95      0.94       357

    accuracy                           0.92       569
   macro avg       0.92      0.91      0.91       569
weighted avg       0.92      0.92      0.92       569



In [56]:
print(classification_report(target_test, target_test_predict))

              precision    recall  f1-score   support

           0       0.95      0.84      0.89        68
           1       0.87      0.96      0.91        75

    accuracy                           0.90       143
   macro avg       0.91      0.90      0.90       143
weighted avg       0.91      0.90      0.90       143



#### 서비스 배포

In [57]:
import pickle

In [58]:
with open('../../../datasets/BreastCancerWisconsin_regression_quest.pkl', 'wb') as regression_file :
    pickle.dump(obj=model, file=regression_file)
    pass